In [2]:
pip install tulipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tulipy: filename=tulipy-0.4.0-cp310-cp310-linux_x86_64.whl size=289864 sha256=3a170a55c46ed7e1ed3c1bf55f7b524837021e48f7f93e6e148ae2f07ee3a9f5
  Stored in directory: /root/.cache/pip/wheels/0a/f1/a7/d402b06d5d6c3fb6270a782341d4753dcd963a00fdde512b07
Successfully built tulipy


In [3]:
# @title Varsayılan başlık metni { run: "auto" }
import pandas as pd
import numpy as np
import tulipy as ti
import yfinance as yf
from datetime import datetime

# Veriyi indir
start = "2022-01-01"
end = datetime.now().strftime('%Y-%m-%d')  # Bugünün tarihini al
data = yf.download(["BTC-USD"], start=start, end=end)

# RSI hesapla
rsi_period = 14
rsi_values = ti.rsi(data['Close'].values, period=rsi_period)

# RSI değerlerini bir pandas Serisi'ne dönüştür ve data['RSI'] sütununa ata
data['RSI'] = pd.Series(rsi_values, index=data.index[-len(rsi_values):])

# Al/Sat sinyallerini oluştur
data['Buy_Signal'] = (data['RSI'] < 30)  # RSI 30'un altındaysa al
data['Sell_Signal'] = (data['RSI'] > 70)  # RSI 70'in üzerindeyse sat

# Sadece 'Buy_Signal' veya 'Sell_Signal' sütunları True olan satırları filtrele
filtered_data = data[(data['Buy_Signal']) | (data['Sell_Signal'])]

print(filtered_data)


[*********************100%%**********************]  1 of 1 completed

                    Open          High           Low         Close  \
Date                                                                 
2022-01-15  43101.898438  43724.671875  42669.035156  43177.398438   
2022-01-16  43172.039062  43436.808594  42691.023438  43113.878906   
2022-01-17  43118.121094  43179.390625  41680.320312  42250.550781   
2022-01-18  42250.074219  42534.402344  41392.214844  42375.632812   
2022-01-19  42374.039062  42478.304688  41242.914062  41744.328125   
...                  ...           ...           ...           ...   
2024-03-10  68500.257812  70005.203125  68239.976562  69019.789062   
2024-03-11  69020.546875  72850.710938  67194.882812  72123.906250   
2024-03-12  72125.125000  72825.656250  68728.851562  71481.289062   
2024-03-13  71482.117188  73637.476562  71334.093750  73083.500000   
2024-03-14  73079.375000  73750.070312  68563.023438  71396.593750   

               Adj Close       Volume        RSI  Buy_Signal  Sell_Signal  
Date         